# Nesse projeto busco realizar um estudo sobre a atual situação do Brasil no cenário da pandemia. Irei usar as séries temporais para realizar a previsão dos números para os próximos 15 dias!

Vamos iniciar realizando as importações das bibliotecas:

In [ ]:
import numpy as np 
import matplotlib.pyplot as plt 
import pandas as pd
import seaborn as sns

Agora o carregamento da base de dados e buscando apenas os números do país:

In [ ]:
covid = pd.read_csv('../input/novel-corona-virus-2019-dataset/covid_19_data.csv')
df_BR = covid[covid['Country/Region'] == 'Brazil']
df_BR

Podemos ver que temos ``ObservationDate``, a coluna onde mostra a data em que foi registrado tal número. Será uma coluna importante para futuras análises na linha do tempo do início da pandemia até os dias mais recentes. ``Province/State`` é o Estado do Brasil onde foram feitos os registros. ``Last Update`` é a data da última atualização da COVID-19. Por fim, ``Confirmed``, ``Deaths`` e ``Recovered`` significam casos confirmados, mortes e recuperados, respectivamente. 

Podemos ver na tabela que os casos começaram a aparecer no dia 26/02/2020, onde foi **confirmado** o primeiro caso de COVID-19 no Brasil, infelizmente não se tem informações sobre em qual região do país. Vamos visualizar esses números de forma gráfica, começando pelos Confirmados:

In [ ]:
sns.lineplot(df_BR['ObservationDate'], df_BR['Confirmed']);

Interessante que quando o número de confirmados ultrapassa a marca dos 250.000 infectados, há um queda brusca, chegando quase a 0. Após a grande linha de queda, voltam a crescer as pessoas infectadas, em um ritmo menor e mais controlado do que na primeira subida da linha. O número de confirmados no país tende a se manter crescendo, porém em números mais baixo e de maneira mais lenta do que no início da pandemia, como mostrado pelo gráfico. É bem provável que essa diminuição atenuante do número de confirmados seja por causa da medida tomada pelo então presidente Jair Bolsonaro, de [não divulgar os números da COVID-19](https://noticias.uol.com.br/ultimas-noticias/deutschewelle/2020/06/06/governo-deixa-de-divulgar-total-de-mortos-e-casos-de-covid-19.htm). [O Brasil é um destaque negativo](https://www.bbc.com/portuguese/brasil-52967730) quando o assunto é o novo vírus, a começar pela [troca de ministros](https://veja.abril.com.br/politica/queda-de-dois-ministros-em-apenas-um-mes-deixa-saude-a-deriva/), uma prova da instabilidade e total desgaste do atual governo. Essas implicações só tornam mais difícil a batalha contra a COVID-19, a própria descrença do presidente, quanto a fatalidade da doença, demonstra a incapacidade das principais autoridades de tomar decisões e criar medidas para controlar o avanço da pandemia. 

In [ ]:
sns.lineplot(df_BR['ObservationDate'], df_BR['Deaths']);

O gráfico acima também não é diferente, há a linha característica mas aqui notamos que o ritmo do avanço das mortes é menor. Podemos perceber que no início há uma rápida escalada até a marca dos mais de 17500 mortos, que logo depois da queda volta a crescer de uma maneira mais controlada e com números mais baixos, se comparados ao crescimento elevado dos confirmados, visto anteriormente. Mas infelizmente o gráfico segue em alta, mostrando que mesmo em um ritmo mais controlado, o número de mortos pela COVID-19 não para de subir.

In [ ]:
sns.lineplot(df_BR['ObservationDate'], df_BR['Recovered']);

Felizmente o número de recuperados também cresce, em um ritmo até mais acelerado que o de mortos. Os índices positivos nos dão maior noção de que possivelmente o número de mortos esteja mais contido pois há casos crescentes de infectados que se recuperaram, ou seja, tudo indica que há menos pessoas morrendo de COVID-19 e mais pessoas curadas. É como se os números de mortos estivessem sendo "trocados" pelo o de recuperados, justificando os gráficos e os avanços em ritmos diferentes. 

In [ ]:
df_sum = df_BR.groupby('ObservationDate').agg({'Confirmed': 'sum', 'Deaths': 'sum', 'Recovered': 'sum'}).reset_index()
plt.stackplot(df_sum['ObservationDate'], [df_sum['Confirmed'], df_sum['Deaths'], df_sum['Recovered']], labels = ['Confirmados', 'Mortos', 'Recuperados'])
plt.legend(loc = 'upper left')

Esse último gráfico comprova o que foi colocado anteriormente sobre o número de recuperados ultrapassar o de mortos. É explícito que há mais casos de confirmados e recuperados do que de mortos. Apesar de tudo, os números continuam crescendo e estão bem altos, indicando que mesmo com o avanço da doença estando mais controlado, não param de crescer os números de pessoas que acabam infectadas e mortas pelo vírus da COVID-19. Já se passaram meses desde o inicio da pandemia e o cenário deveria ser outro, mas também sabemos que as medidas tomadas não foram as mais eficazes e que muito brasileiros não se cuidam como o recomendado. Vários estabelecimentos ficaram abertos mesmo quando não deveriam, pessoas que não saem de máscara, não se higienizam corretamente e para piorar, não respeitam a distância recomendada nos espaços públicos. 

# Séries Temporais para a previsão dos números da COVID-19 no Brasil

### **Previsão do número de CONFIRMADOS para os próximos 15 dias:**

In [ ]:
confirmados = pd.read_csv('../input/novel-corona-virus-2019-dataset/time_series_covid_19_confirmed.csv')
confirmados = confirmados[confirmados['Country/Region'] == 'Brazil']
confirmados

Vamos ver as colunas onde estão as dadas para assim realizar as previsões:

In [ ]:
columns1 = confirmados.keys()
columns1

A partir da quarta coluna tem as datas, vamos guardar somente elas, que é de fato o que iremos trabalhar:

In [ ]:
confirmados = confirmados.loc[:, columns1[4]:columns1[-1]]
confirmados

Vamos criar a variável ``dates1`` para armazenar as datas com os números de confirmados:

In [ ]:
dates1 = confirmados.keys()
y_c = []
for i in dates1:
    y_c.append(confirmados[i].sum())

Vamos fazer o ``reshape`` para que o modelo possa fazer corretamente a previsão. Ele tem que estar em formato de matriz e é isso que vamos fazer agora:

In [ ]:
y_c = np.array(y_c).reshape(-1,1)
y_c 

Agora é a vez da variável ``x_c``, nela será armazenada os dias que já se passaram desde o inicio dessa base de dados (dia 22/01). 

In [ ]:
x_c = np.arange(len(dates1)).reshape(-1,1)
x_c

Nós temos 217 dias, que são aproximadamente 7 meses de dados para serem avaliados até a data de hoje, dia 28/08/2020.

E para realizar a previsão dos próximos 15 dias, será criado o ``forecast_c``, terá 15 dias a mais que a variável ``x_c``:

In [ ]:
forecast_c = np.arange(len(dates1) + 15).reshape(-1,1)

**Criando o modelo e o avaliando:**

In [ ]:
from sklearn.model_selection import train_test_split
X_train_c, X_test_c, Y_train_c, Y_test_c = train_test_split(x_c, y_c, test_size = 0.40, shuffle = False)

In [ ]:
from sklearn.preprocessing import PolynomialFeatures
poly_c = PolynomialFeatures(degree = 3)
X_train_poly_c = poly_c.fit_transform(X_train_c)
X_test_poly_c = poly_c.transform(X_test_c)

In [ ]:
from sklearn.linear_model import LinearRegression
lr_c = LinearRegression()
lr_c.fit(X_train_poly_c, Y_train_c)

In [ ]:
poly_pred_c = lr_c.predict(X_test_poly_c)
plt.plot(poly_pred_c, linestyle = 'dashed')
plt.plot(Y_test_c)

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error
print('MAE:', mean_absolute_error(poly_pred_c, Y_test_c))
print('MSE:', mean_squared_error(poly_pred_c, Y_test_c))
print('RMSE:', np.sqrt(mean_absolute_error(poly_pred_c, Y_test_c)))

Esse gráfico representa a avaliação do meu modelo, em laranja são os números verdadeiros e o pontilhado é a previsão que o modelo fez. Quanto menor o número do ``RMSE`` melhor! As duas linhas estão bem próximas, o que mostra uma avaliação positiva pois coloca meu modelo prevendo de maneira próxima os números reais. 

### **Previsão do número de MORTES para os próximos 15 dias:**

Serão realizadas as mesmas etapas que foram feitas anteriormente!

In [ ]:
mortes = pd.read_csv('../input/novel-corona-virus-2019-dataset/time_series_covid_19_deaths.csv')
mortes = mortes[mortes['Country/Region'] == 'Brazil']
mortes

In [ ]:
columns = mortes.keys()
mortes = mortes.loc[:, columns[4]:columns[-1]]
mortes

In [ ]:
dates = mortes.keys()
y = []
for i in dates:
    y.append(mortes[i].sum())

In [ ]:
y = np.array(y).reshape(-1,1)
y

In [ ]:
x = np.arange(len(dates)).reshape(-1,1)
x

In [ ]:
forecast = np.arange(len(dates) + 15).reshape(-1,1)

Criando o modelo e o avaliando:

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(x, y, test_size = 0.30, shuffle = False)

In [ ]:
poly = PolynomialFeatures(degree = 2)
X_train_poly = poly.fit_transform(X_train)
X_test_poly = poly.transform(X_test)

In [ ]:
lr = LinearRegression()
lr.fit(X_train_poly, Y_train)

In [ ]:
poly_pred = lr.predict(X_test_poly)
plt.plot(poly_pred, linestyle = 'dashed')
plt.plot(Y_test)

In [ ]:
print('MAE:', mean_absolute_error(poly_pred, Y_test))
print('MSE:', mean_squared_error(poly_pred, Y_test))
print('RMSE:', np.sqrt(mean_absolute_error(poly_pred, Y_test)))

A avaliação do modelo foi bem positiva, as previsões chegam bem perto dos valores reais. E o valor da ``RMSE`` é bem pequeno, o que indica uma média de erros baixa.

### **Previsão do número de RECUPERADOS para os próximos 15 dias:**

In [ ]:
recuperados = pd.read_csv('../input/novel-corona-virus-2019-dataset/time_series_covid_19_recovered.csv')
recuperados = recuperados[recuperados['Country/Region'] == 'Brazil']
recuperados

In [ ]:
columns2 = recuperados.keys()
recuperados = recuperados.loc[:, columns2[4]:columns2[-1]]
recuperados

In [ ]:
dates2 = recuperados.keys()
y_r = []
for i in dates2:
    y_r.append(recuperados[i].sum())

In [ ]:
y_r = np.array(y_r).reshape(-1,1)
y_r

In [ ]:
x_r = np.arange(len(dates2)).reshape(-1,1)
x_r

In [ ]:
forecast_r = np.arange(len(dates2) + 15).reshape(-1,1)

Criando o modelo e o avaliando

In [ ]:
X_train_r, X_test_r, Y_train_r, Y_test_r = train_test_split(x_r, y_r, test_size = 0.30, shuffle = False)

In [ ]:
poly_r = PolynomialFeatures(degree = 3)
X_train_poly_r = poly_r.fit_transform(X_train_r)
X_test_poly_r = poly_r.transform(X_test_r)

In [ ]:
lr_r = LinearRegression()
lr_r.fit(X_train_poly_r, Y_train_r)

In [ ]:
poly_pred_r = lr_r.predict(X_test_poly_r)
plt.plot(poly_pred_r, linestyle = 'dashed')
plt.plot(Y_test_r)

In [ ]:
print('MAE:', mean_absolute_error(poly_pred_r, Y_test_r))
print('MSE:', mean_squared_error(poly_pred_r, Y_test_r))
print('RMSE:', np.sqrt(mean_absolute_error(poly_pred_r, Y_test_r)))

Vale lembrar que a quantidade de registros é diferente, por isso o valores dos ``RMSE`` variam. Lembrando que a variável ``mortes`` tem um número menor de casos registrados. Voltando para os gráficos iniciais, a quantidade de pessoas que são confirmadas diaraiamente com o novo vírus é maior do que o número de pessoas que morrem por causa dele, assim como o número de infectados que se recuperam também é bem maior que o número de infectados que morrem por causa da doença. Isso justifica a diferença entre os ``RMSE``, que é a média de erros.

# PREVISÃO PARA OS PRÓXIMOS 15 DIAS
Aqui iremos utilizar as variáveis ``forecast``, ``forecast_c`` e ``forecast_r`` para realizar as previsões das séries temporais para próximos 15 dias

### Próximos 15 dias do número de confirmados:

In [ ]:
X_train_all_c = poly_c.transform(forecast_c)
pred_all_c = lr_c.predict(X_train_all_c)

plt.plot(forecast_c[:-15], y_c, color='red')
plt.plot(forecast_c, pred_all_c, linestyle='dashed')
plt.title('Casos Confirmados de COVID-19')
plt.xlabel('Dias desde 22/01/2020')
plt.ylabel('Número de confirmados')
plt.legend(['Casos Confirmados', 'Previsões']);

A linha pontilhada ultrapassa a vermelha pois estão sendo previstos os 15 dias futuros. E como é mostrado, a previsão para os próximos 15 dias é um crescimento do número de confirmados. Justamente o que foi discutido anteriormente, infelizmente as medidas não foram tão eficazes e muitos brasileiros também não seguem as orientações que são recomendadas quando necessário sair de casa. Se continuar dessa maneira, sem medidas mais rígidas e a falta de contribuição das pessoas, o virus irá continuar a se espalhar e afetar mais pessoas.

### Próximos 15 dias do número de mortes:

In [ ]:
X_train_all = poly.transform(forecast)
pred_all = lr.predict(X_train_all)

plt.plot(forecast[:-15], y, color='red')
plt.plot(forecast, pred_all, linestyle='dashed')
plt.title('Mortes por COVID-19')
plt.xlabel('Dias desde 22/01/2020')
plt.ylabel('Número de mortes')
plt.legend(['Mortes', 'Previsões']);

Infelizmente a previsão para os próximos 15 dias é que continue a crescer o número de mortes no país. Pelo o que o gráfico indica e já foi comentado anteriormente, é bem possível que o número de mortes se mantenha alto porém controlado. O número de pessoas que morrem em decorrência do vírus ainda é mais baixo que os outros dados analisados, porém são valores elevados e preocupantes, mesmo que se mantenha estagnado e num ritmo menor que a taxa de recuperados, o ideal seria se esse gráfico fosse decrescente e tendendo rapidamente para o zero. 

### Próximos 15 dias do número de recuperados:

In [ ]:
X_train_all_r = poly_r.transform(forecast_r)
pred_all_r = lr_r.predict(X_train_all_r)

plt.plot(forecast_r[:-15], y_r, color='red')
plt.plot(forecast_r, pred_all_r, linestyle='dashed')
plt.title('Recuperados da COVID-19')
plt.xlabel('Dias desde 22/01/2020')
plt.ylabel('Número de recuperados')
plt.legend(['Recuperados', 'Previsões']);

Para os próximos 15 dias a previsão é que cresça o número de pessoas que irão se curar da COVID-19. Felizmente se mantém em um bom ritmo, o que indica que muitos dos infectados mais se recuperam do que morrem. O ideal é que a curva se mantenha sempre em alta e superando o número de mortes. Mesmo com a retomada de algumas atividades, é possível que esse número se mantenha elevado. Como já tratado antes, seguir as recomendações, por mais simples que pareçam, realmente auxiliam no combate contra a doença e ajuda a manter essa linha sempre crescente e decrescendo a linha de confirmados e consequentemente a de mortes.  